In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/datasets/train /content/train

## **1. Data Augmentation**


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Initial transform: Resize and convert to tensor only
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ensure all images are the same size
    transforms.ToTensor()  # Convert images to tensor format
])

# Load dataset with simple transform (no normalization)
train_dataset_simple = datasets.ImageFolder(root='/content/train', transform=simple_transform)

# Create a DataLoader for the training data
train_loader_simple = DataLoader(train_dataset_simple, batch_size=32, shuffle=False, num_workers=2)

def compute_mean_std(loader):
    mean = 0.0
    std = 0.0
    total_images_count = 0
    for images, _ in loader:
        batch_samples = images.size(0)  # Number of images in the batch
        images = images.view(batch_samples, images.size(1), -1)  # Flatten the images
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count
    return mean, std

# Calculate mean and standard deviation
mean, std = compute_mean_std(train_loader_simple)

print(f"Mean: {mean}, Std: {std}")


Mean: tensor([0.4823, 0.4823, 0.4823]), Std: tensor([0.2216, 0.2216, 0.2216])


In [ ]:
# Use the computed mean and std values and perform data augmentation on train dataset
train_transform = transforms.Compose([
       transforms.RandomRotation(10),  # Small rotations for generalization
       transforms.RandomAffine(degrees=0, translate=(0.03, 0.03)),  # Small translations
       transforms.RandomResizedCrop(224, scale=(0.95, 1.05)),  # Slight zoom
       transforms.ToTensor(),  # Convert to tensor
       transforms.Normalize(mean=mean.tolist(), std=std.tolist())  # Use computed mean and std
   ])

#Only resize and normalize test dataset
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())  # Same normalization for test set
])

!cp -r /content/drive/MyDrive/datasets/test /content/test

# Load dataset (assumes dataset is in a directory with subfolders for each class)
train_dataset = datasets.ImageFolder(root='/content/train', transform=train_transform)
test_dataset = datasets.ImageFolder(root='/content/test', transform=test_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers= 2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check class to index mapping
print(train_dataset.class_to_idx)  # {'a_normal_xrays': 0, 'bacterial pneumonia': 1, 'viral pneumonia': 2}


{'a_normal_xrays': 0, 'bacterial pneumonia': 1, 'viral pneumonia': 2}


## **2. Data loaded with augmentation methods. Load/define Vgg16 model for transfer learning**

In [ ]:
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Load a pretrained VGG16 model
model = models.vgg16(pretrained=True)

# Modify the classifier (for your specific number of classes, e.g., 2 classes for binary classification)
model.classifier[6] = nn.Linear(4096, 3)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Freeze all layers except the classifier
for param in model.features.parameters():
    param.requires_grad = False

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Train for 10 epochs
num_epochs = 10



## **3. Training**

In [ ]:
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        # Move images and labels to the device (GPU)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average loss for the epoch
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}')

    # Step the scheduler based on the average loss
    scheduler.step(avg_loss)

    # Log the current learning rate
    current_lr = scheduler.get_last_lr()
    print(f"Current Learning Rate: {current_lr}")


print('Training finished.')

# Save the final model state after all epochs
torch.save(model.state_dict(), 'VGG16_on_xray_weights_data_augmentation1.pth')
torch.save(model.state_dict(), '/content/drive/MyDrive/VGG16_on_xray_weights_data_augmentation1.pth')

Epoch [1/10], Loss: 0.7997261817835591
Current Learning Rate: [0.001]
Epoch [2/10], Loss: 0.7005845346699463
Current Learning Rate: [0.001]
Epoch [3/10], Loss: 0.6579501560312108
Current Learning Rate: [0.001]
Epoch [4/10], Loss: 0.5694618964670626
Current Learning Rate: [0.001]
Epoch [5/10], Loss: 0.5455121008530717
Current Learning Rate: [0.001]
Epoch [6/10], Loss: 0.5089866733806996
Current Learning Rate: [0.001]
Epoch [7/10], Loss: 0.5174218904387
Current Learning Rate: [0.001]
Epoch [8/10], Loss: 0.49219228564961554
Current Learning Rate: [0.001]
Epoch [9/10], Loss: 0.49413634544135604
Current Learning Rate: [0.001]
Epoch [10/10], Loss: 0.4880158719292448
Current Learning Rate: [0.001]
Training finished.


In [ ]:
# Train for an additional 20 epochs at .0001 lr

# Load the saved model state to resume training
model.load_state_dict(torch.load('VGG16_on_xray_weights_data_augmentation1.pth', weights_only = True))

# Freeze all layers except the classifier (if not already done)
for param in model.features.parameters():
    param.requires_grad = False

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Continue training for another 20 epochs
num_epochs = 20  # Adjust the number of additional epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        # Move images and labels to the device (GPU)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average loss for the epoch
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}')

    # Step the scheduler after each epoch, based on the average loss
    scheduler.step(avg_loss)

print('Additional training finished.')

# Save the final model state after all epochs
torch.save(model.state_dict(), 'VGG16_on_xray_weights_data_augmentation1.pth')
torch.save(model.state_dict(), '/content/drive/MyDrive/VGG16_on_xray_weights_data_augmentation1.pth')

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/20], Loss: 0.39685526188531534
Epoch [2/20], Loss: 0.368242229100751
Epoch [3/20], Loss: 0.3466511806644545
Epoch [4/20], Loss: 0.35579090263763086
Epoch [5/20], Loss: 0.3460860557717048
Epoch [6/20], Loss: 0.3337754714159878
Epoch [7/20], Loss: 0.32607292050232917
Epoch [8/20], Loss: 0.32333679853772823
Epoch [9/20], Loss: 0.31555928134479405
Epoch [10/20], Loss: 0.3192964631355613
Epoch [11/20], Loss: 0.31350377243172173
Epoch [12/20], Loss: 0.30251902666377145
Epoch [13/20], Loss: 0.293893978456778
Epoch [14/20], Loss: 0.2960901893446782
Epoch [15/20], Loss: 0.29379256764438255
Epoch [16/20], Loss: 0.28939124425313223
Epoch [17/20], Loss: 0.28701969094437324
Epoch [18/20], Loss: 0.27816906740511854
Epoch [19/20], Loss: 0.2758285485527998
Epoch [20/20], Loss: 0.2642974432534967
Additional training finished.


In [ ]:
import torch.nn.functional as F


# Function to evaluate the model on the test dataset
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode (no gradient computation)
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for images, labels in test_loader:
            # Move images and labels to the device (GPU)
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Example usage after training
evaluate_model(model, test_loader)

Test Accuracy: 82.21%


### **4. Additional Training**

In [ ]:
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Step 1: Initialize the model architecture with the correct number of classes
num_classes = 3  # Replace with the actual number of classes in your dataset
model = models.vgg16(pretrained=False, num_classes=num_classes)

# Step 2: Load the saved weights from previous training session
model.load_state_dict(torch.load('/content/drive/MyDrive/VGG16_on_xray_weights_data_augmentation1.pth'))

# Freeze all layers except the classifier
for param in model.features.parameters():
    param.requires_grad = False

# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set the model to training mode
model.train()

# Define same loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # loss had not increased twice consecutively, so setting lr as same value

# Define learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-6-d0ea4be60162>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

In [ ]:
# Continue training for another 20 epochs
num_epochs = 20

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        # Move images and labels to the device (GPU)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average loss for the epoch
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}')

    # Step the scheduler after each epoch, based on the average loss
    scheduler.step(avg_loss)

print('Additional training finished.')

# Save the final model state after all epochs, save in different file in case
torch.save(model.state_dict(), 'VGG16_on_xray_weights_data_augmentation2.pth')
torch.save(model.state_dict(), '/content/drive/MyDrive/VGG16_on_xray_weights_data_augmentation2.pth')

Epoch [1/20], Loss: 0.2731098582042507
Epoch [2/20], Loss: 0.2554500456328041
Epoch [3/20], Loss: 0.24734452485672534
Epoch [4/20], Loss: 0.25171911821592075
Epoch [5/20], Loss: 0.24728679871815115
Epoch [6/20], Loss: 0.22475284792802816
Epoch [7/20], Loss: 0.23835025471778004
Epoch [8/20], Loss: 0.21908789251479635
Epoch [9/20], Loss: 0.22775914949690637
Epoch [10/20], Loss: 0.20906987347485828
Epoch [11/20], Loss: 0.21901801494778667
Epoch [12/20], Loss: 0.2016317397813124
Epoch [13/20], Loss: 0.19505999561833456
Epoch [14/20], Loss: 0.2059084623137866
Epoch [15/20], Loss: 0.19688985032812217
Epoch [16/20], Loss: 0.18734390441716814
Epoch [17/20], Loss: 0.19018937775328115
Epoch [18/20], Loss: 0.18963840091886697
Epoch [19/20], Loss: 0.171672854126871
Epoch [20/20], Loss: 0.16703784030410776
Additional training finished.


In [ ]:
#Evaluate once more on test set
import torch.nn.functional as F


# Function to evaluate the model on the test dataset
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode (no gradient computation)
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for images, labels in test_loader:
            # Move images and labels to the device (GPU)
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Example usage after training
evaluate_model(model, test_loader)

Test Accuracy: 80.61%


Not seeing further improvement in test accuracy. Potentially overfitting training set
